# How to do optimization better than "classical" mathematician? (scipy.optimize)

We will use <a href= "https://docs.scipy.org/doc/scipy/reference/optimize.html#module-scipy.optimize"> scipy.optimize </a> package for black-box optimization: we do not rely on the mathematical expression of the function that we are optimizing. Note that this expression can often be used for more efficient, non black-box, optimization.
<b>Prerequisites:</b>
    1. Numpy, Scipy
    2. matplotlib

In [5]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

# How to start: get to know your problem

Not all optimization problems are equal. Knowing your problem enables you to choose the right tool. Remember to account for:
    
    1. dimensionality of the problem (why?);
    2. convex vs non-convex optimization (what is easier?);
    3. smooth vs non-smooth problems (what is easier?); 
    4. noisy vs exact data;

Characterise each of the below functions and mention which of them are easier to optimize:
<img src="images/noisy.png"> 
<img src="images/nonsmooth.png"> 
<img src="images/smooth.png"> 
<img src="images/convex.png"> 
<img src="images/non-convex.png"> 

# What is available? Overview of different optimizers

## 0. Getting started: 1D optimization

Use scipy.optimize.brent() to minimize 1D functions. It combines a bracketing strategy (golden section) with a parabolic approximation.

In [3]:
from scipy import optimize
def f(x):
    return -np.exp(-(x - .7)**2)
x_min = optimize.brent(f)  # It actually converges in 9 iterations!
x_min

0.6999999997839409

In [4]:
def main():
    x = np.linspace(-1, 2, 100)
    y = -np.exp(-(x - .7)**2)

    plt.figure()
    plt.plot(x, y)
    plt.xlabel('$x$')
    plt.ylabel('$\exp(-(x-0.7)^2)$')

    plt.show()
    
if __name__ == '__main__':
    main()

We consider the Rosenbrock's function, which designed specifically for testing optimization techniques, it has curved, narrow valley.
<img src="images/rosenbock.png"> 


## 1. Gradient based methods: Conjugate gradient descent

Gradient descent basically consists in taking small steps in the direction of the gradient, that is the direction of the steepest descent.

Methods based on conjugate gradient are named with ‘cg’ in scipy. The simple conjugate gradient method to minimize a function is ``` scipy.optimize.fmin_cg():```

In [6]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
optimize.fmin_cg(f, [2, 2])  

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 120
         Gradient evaluations: 30


array([ 0.99998968,  0.99997855])

These methods need the gradient of the function. They can compute it, but will perform better if you can pass them the gradient:

In [21]:
def fprime(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))
optimize.fmin_cg(f, [2, 2], fprime=fprime)  

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 13
         Function evaluations: 30
         Gradient evaluations: 30


array([ 0.99999199,  0.99998336])

Note that the function has only been evaluated 30 times, compared to 120 without the gradient.

## 2. Gradient based methods: Newton and quasi-newton methods

Newton methods use a local quadratic approximation to compute the jump direction. For this purpose, they rely on the 2 first derivative of the function: the gradient and the Hessian.

In scipy, the Newton method for optimization is implemented in ``` scipy.optimize.fmin_ncg()``` (cg here refers to that fact that an inner operation, the inversion of the Hessian, is performed by conjugate gradient). ``` scipy.optimize.fmin_tnc()``` can be use for constraint problems, although it is less versatile:

In [22]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
def fprime(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))
optimize.fmin_ncg(f, [2, 2], fprime=fprime)    

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 51
         Hessian evaluations: 0


array([ 1.,  1.])

Note that compared to a conjugate gradient (above), Newton’s method has required less function evaluations, but more gradient evaluations, as it uses it to approximate the Hessian. Let’s compute the Hessian and pass it to the algorithm:

In [23]:
def hessian(x): # Computed with sympy
    return np.array(((1 - 4*x[1] + 12*x[0]**2, -4*x[0]), (-4*x[0], 2)))
optimize.fmin_ncg(f, [2, 2], fprime=fprime, fhess=hessian)   

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 19
         Hessian evaluations: 9


array([ 1.,  1.])

<b>Note:</b> At very high-dimension, the inversion of the Hessian can be costly and unstable (large scale > 250).

<b>Note:</b> Newton optimizers should not to be confused with Newton’s root finding method, based on the same principles, ``` scipy.optimize.newton().```

<b>BFGS:</b> BFGS (Broyden-Fletcher-Goldfarb-Shannon algorithm) refines at each step an approximation of the Hessian.

In [24]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
def fprime(x):
    return np.array((-2*.5*(1 - x[0]) - 4*x[0]*(x[1] - x[0]**2), 2*(x[1] - x[0]**2)))
optimize.fmin_bfgs(f, [2, 2], fprime=fprime)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 16
         Function evaluations: 24
         Gradient evaluations: 24


array([ 1.00000017,  1.00000026])

## 3. Gradient-less methods: Simplex method (the Nelder-Mead)

The simplex algorithm is probably the simplest way to minimize a fairly well-behaved function. It requires only function evaluations and is a good choice for simple minimization problems. However, because it does not use any gradient evaluations, it may take longer to find the minimum.

The Nelder-Mead algorithms is a generalization of dichotomy approaches to high-dimensional spaces. The algorithm works by refining a simplex, the generalization of intervals and triangles to high-dimensional spaces, to bracket the minimum.

Strong points: it is robust to noise, as it does not rely on computing gradients. Thus it can work on functions that are not locally smooth such as experimental data points, as long as they display a large-scale bell-shape behavior. However it is slower than gradient-based methods on smooth, non-noisy functions.

In scipy, ```scipy.optimize.fmin()``` or minimize routine with ```method='nelder-mead'``` implements the Nelder-Mead approach:

In [12]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
optimize.fmin(f, [2, 2])

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 46
         Function evaluations: 91


array([ 0.99998568,  0.99996682])

Another optimization algorithm that needs only function calls to find the minimum is Powell‘s method available by setting ```method='powell'``` in minimize.

## 4. Global optimizers

If your problem does not admit a unique local minimum (which can be hard to test unless the function is convex), and you do not have prior information to initialize the optimization close to the solution, you may need a global optimizer.

``` scipy.optimize.brute()``` evaluates the function on a given grid of parameters and returns the parameters corresponding to the minimum value. The parameters are specified with ranges given to ``` numpy.mgrid```. By default, 20 steps are taken in each direction:

In [26]:
def f(x):   # The rosenbrock function
    return .5*(1 - x[0])**2 + (x[1] - x[0]**2)**2
optimize.brute(f, ((-1, 2), (-1, 2)))

array([ 1.00001462,  1.00001547])

## 5. Practical guide to optimization with scipy

<b>Without knowledge of the gradient:</b>
* In general, prefer ``` BFGS (scipy.optimize.fmin_bfgs())```, even if you have to approximate numerically gradients
* On well-conditioned problems, gradient-free methods, work well in high dimension, but they collapse for ill-conditioned problems.

***

<b>With knowledge of the gradient:</b>
	
* Use BFGS ``` (scipy.optimize.fmin_bfgs())```.
* Computational overhead of BFGS is larger than that of conjugate gradient. On the other side, BFGS usually needs less function evaluations than CG. Thus conjugate gradient method is better than BFGS at optimizing computationally cheap functions.

***

<b>With knowledge of the Hessian:</b>

* If you can compute the Hessian, prefer the Newton method ``` (scipy.optimize.fmin_ncg()).```

***

<b>If you have noisy measurements:</b>

* Use gradient-free methods like simplex method ```(scipy.optimize.fmin()).```